In [1]:
from TCs_class_methods import TCs, Testwarn
from aqua.util import load_yaml

Load configuration file with all the tempest options

In [2]:
tdict = load_yaml('config/config_levante.yml')
tdict

{'dataset': {'model': 'IFS', 'exp': 'tco2559-ng5'},
 'paths': {'regdir': '/scratch/b/b382216/regrid_new',
  'tmpdir': '/scratch/b/b382216/tmpdir_new',
  'fulldir': '/scratch/b/b382216/fullres_new'},
 'grids': {'lowgrid': 'r100', 'highgrid': 'r010'},
 'varlist': ['msl', '10u', '10v', 'tp'],
 'time': {'start_date': '2020-08-01',
  'end_date': '2020-09-30',
  'frequency': '6h'},
 'detect': {'boxdim': 10},
 'stitch': {'n_days_freq': 25, 'n_days_ext': 15}}

Initialise the class with the dictionary from the YAML file

In [3]:
warnings = Testwarn(loglevel='ERROR')
tropical = TCs(tdict=tdict, streaming=True, stream_step=3, stream_unit="days", stream_startdate=tdict['time']['start_date'], loglevel = "WARNING")
#tropical = TCs(paths=tdict['paths'], var2store=tdict['varlist'])

Run the code on a loop on the detect nodes

In [4]:
tropical.detect_nodes_zoomin(start_date=tdict['time']['start_date'], 
                             end_date=tdict['time']['end_date'], 
                             frequency=tdict['time']['frequency'])

2023-04-13 11:37:23 :: TCs :: WARNING  -> Initialised streaming for 3 days starting on 2020-08-01
2023-04-13 11:38:01 :: TCs :: WARNING  -> 20200801T00
2023-04-13 11:38:09 :: TCs :: WARNING  -> 20200801T06
2023-04-13 11:38:17 :: TCs :: WARNING  -> 20200801T12
2023-04-13 11:38:25 :: TCs :: WARNING  -> 20200801T18
2023-04-13 11:38:34 :: TCs :: WARNING  -> 20200802T00
2023-04-13 11:38:41 :: TCs :: WARNING  -> 20200802T06
2023-04-13 11:38:48 :: TCs :: WARNING  -> 20200802T12
2023-04-13 11:38:56 :: TCs :: WARNING  -> 20200802T18
2023-04-13 11:39:05 :: TCs :: WARNING  -> 20200803T00
2023-04-13 11:39:15 :: TCs :: WARNING  -> 20200803T06
2023-04-13 11:39:25 :: TCs :: WARNING  -> 20200803T12
2023-04-13 11:39:35 :: TCs :: WARNING  -> 20200803T18


Run the code for the detect nodes, after having set up the days freq and the extensions days

In [4]:
tropical.stitch_nodes_zoomin(start_date=tdict['time']['start_date'], end_date=tdict['time']['end_date'],
                             n_days_freq=tdict['stitch']['n_days_freq'], n_days_ext=tdict['stitch']['n_days_ext'])

2023-03-30 19:12:52 WARNING: 2020-08-01 00:00:00


msl
2020080306
2020080312
2020080318
2020080400
2020080406
2020080412
2020080418
2020080500
2020080506
2020080512
2020080518
2020080600
2020080606
2020080612
2020080618
2020080700
2020080706
2020080712
2020080718
2020080800
2020080806
2020080812
2020080818
2020080900
2020080906
2020080912
2020080918
2020081000
2020081006
2020081012
2020081018
2020081100
2020081106
2020081112
2020081118
2020081200
2020081206
2020081212
2020081218
2020081300
2020081306
2020081312
2020081318
2020081400
2020081406
2020081412
2020082400
2020082406
2020082412
2020082418
2020082500
2020082506
2020082512
2020082518
2020082600
2020082606
2020082612
2020082618
2020082700
2020082706
2020082712
2020082718
2020082800
2020082806
2020082812
2020082818
2020082900
2020082906
2020082912
2020082918
2020083000
2020083006
2020083012
2020083018
2020083100
2020083106
2020083112
2020083118
2020090100
2020090106
2020090112
2020090118
2020090200
2020090206
2020090212
2020090218
2020090300
2020090306
2020090312
2020090318
202009

2023-03-30 19:14:58 WARNING: 2020-08-26 00:00:00


msl
2020081100
2020081106
2020081112
2020081118
2020081200
2020081206
2020081212
2020081218
2020081300
2020081306
2020081312
2020081318
2020081400
2020081406
2020081412
2020082400
2020082406
2020082412
2020082418
2020082500
2020082506
2020082512
2020082518
2020082600
2020082606
2020082612
2020082618
2020082700
2020082706
2020082712
2020082718
2020082800
2020082806
2020082812
2020082818
2020082900
2020082906
2020082912
2020082918
2020083000
2020083006
2020083012
2020083018
2020083100
2020083106
2020083112
2020083118
2020090100
2020090106
2020090112
2020090118
2020090200
2020090206
2020090212
2020090218
2020090300
2020090306
2020090312
2020090318
2020090400
2020090406
2020090412
2020090418
2020090500
2020090506
2020090512
2020090518
2020090600
2020090606
2020090612
2020090618
2020090700
2020090706
2020090800
2020090806
2020090812
2020090818
2020090900
2020090906
2020090912
2020090918
2020091000
2020091006
2020091012
2020091018
2020091100
2020091106
2020091112
2020091118
2020091200
202009